# CX 4230, Spring 2016 [19]: Generating Numbers Pseudorandomly

The discrete event simulator needs a high-quality random number generator. Constructing these is a tricky business! However, it's good to know some of the key principles underlying them, which are rooted in "bread-and-butter" discrete math and computer science (namely, number theory).

Here are some references for today's notebook:
* Slides from today's class: [link](https://t-square.gatech.edu/access/content/group/gtc-59b8-dc03-5a67-a5f4-88b8e4d5b69a/cx4230-sp16--19-pseudorandom-lcg.pdf)
* Lemmis & Park (2006): [link](https://t-square.gatech.edu/access/content/group/gtc-59b8-dc03-5a67-a5f4-88b8e4d5b69a/Lemmis-Park-2006--des-first-course.pdf)
* Knuth (2014): [link](https://t-square.gatech.edu/access/content/group/gtc-59b8-dc03-5a67-a5f4-88b8e4d5b69a/Knuth-TAOCP-v2--9780133488791.pdf)

## Big integers

Python 3 supports _big_ integers, without having to revert to a special type. (By contrast, most languages assume a primitive integral type.) This behavior makes pseudorandom number generation algorithms easy to implement, albeit in a way that might be oblivious to the _actual_ cost of manipulating integers that are larger than the machine's natural word size.

In [1]:
from math import log

def print_with_log2 (x):
    print (x, "~=", "2^(%g)" % log (x, 2), "[%s]" % type (x))
    
x = 1024
print_with_log2 (x)
print_with_log2 (x**2)
print_with_log2 (x**4)
print_with_log2 (x**8)
print_with_log2 (x**16)
print_with_log2 (x**32)

1024 ~= 2^(10) [<class 'int'>]
1048576 ~= 2^(20) [<class 'int'>]
1099511627776 ~= 2^(40) [<class 'int'>]
1208925819614629174706176 ~= 2^(80) [<class 'int'>]
1461501637330902918203684832716283019655932542976 ~= 2^(160) [<class 'int'>]
2135987035920910082395021706169552114602704522356652769947041607822219725780640550022962086936576 ~= 2^(320) [<class 'int'>]


## Linear congruential generators

A _linear congruential sequence (LCS)_ is an integer sequence $X_0$, $X_1$, $\ldots$, $X_n$, generated by the recurrence,

$$
  X_n \leftarrow (a \cdot X_{n-1} + c) \bmod m,
$$

where
* $m > 0$ is the _modulus_ of the sequence;
* $2 \leq a < m$ is the _multiplier_;
* $0 \leq c < m$ is the _increment_; and
* $X_0$ is a given _seed_ or _starting value_ for the sequence.

The right-hand side of $X_n$ is a _linear congruential generator_, or _LCG_.

When $c=0$, the corresponding $LCG$ is also called a _multiplicative congruential method_ or a _Lehmer generator_, the latter being named after the generator's inventor.

> The definition above restricts $a$ to be at least 2. Why? That is, what happens when $a=0$ or $a=1$?

**Exercise.** Implement an LCG. That is, your function should generate $X_{i+1}$ given the previous value, $X_i$, and parameters, $(a, m, c)$, of the LCG.

In [2]:
def g (xi, m, a, c=0):
    """Implements an LCG. The increment defaults to 0."""
    # @YOUSE
    return (a*xi + c) % m

**Exercise.** Given a seed, $X_0$, and parameters of an LCG, $(a, m, c)$, write a function that generates the $m$ values of the LCS starting at $X_0$ and returns them as a list. ($X_0$ should be the first element of that list.) If at any point the current value of the sequence equals $X_0$, this function should terminate at that point and _include_ that value.

In [3]:
def g_seq (x0, m, a, c=0):
    X = [x0]
    
    # @YOUSE
    i, xi = 0, x0
    while (i < m) and (xi!=x0 or i==0):
        i, xi = i+1, g (xi, m, a, c)
        X.append (xi)
        
    # Return the unique elements
    return X

In [4]:
# Test code
SEED = 2
A = 13
M = 2**6
C = 0
X = g_seq (SEED, M, A, C)

print ("SEED ==", SEED)
print ("A ==", A)
print ("M ==", M)
print ("C ==", C)
print ("\n--> X ==", X)

# Some specific test cases
if SEED == 2 and A == 13 and M == 2**6:
    assert X[0] == X[-1] # Must be periodic including X[0]
    assert X == [2, 26, 18, 42, 34, 58, 50, 10, 2]
    print ("\n(Passed an explicit test case.)")

SEED == 2
A == 13
M == 64
C == 0

--> X == [2, 26, 18, 42, 34, 58, 50, 10, 2]

(Passed an explicit test case.)


**Exercise.** Let $m=10$. Use your `g_seq()` function to find a set of values for $X_0$, $a$, and $c$ such that all values are enumerated but the sequence _never_ returns to to $X_0$.

In [5]:
M = 10

# @YOUSE
X0, A, C = 7, 4, 2

# Test code:
X = g_seq (X0, M, A, C)
print (X)

[7, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2]


**Exercise.** Let $m=13$. Compute Lehmer sequences for the following cases:

* $X_0=1$ and $a=6$
* $X_0=1$ and $a=7$
* $X_0=1$ and $a=5$
* $X_0=2$ and $a=5$
* $X_0=4$ and $a=5$

Which are full-period sequences? Which look "the most (uniformly) random" to you? Why or why not?

In [6]:
# @YOUSE: Try out the preceding cases
print ("(x_0=1, m=13, a=6):", g_seq (1, 13, 6))
print ("(x_0=1, m=13, a=7):", g_seq (1, 13, 7))
print ("(x_0=1, m=13, a=5):", g_seq (1, 13, 5))
print ("(x_0=2, m=13, a=5):", g_seq (2, 13, 5))
print ("(x_0=4, m=13, a=5):", g_seq (4, 13, 5))

(x_0=1, m=13, a=6): [1, 6, 10, 8, 9, 2, 12, 7, 3, 5, 4, 11, 1]
(x_0=1, m=13, a=7): [1, 7, 10, 5, 9, 11, 12, 6, 3, 8, 4, 2, 1]
(x_0=1, m=13, a=5): [1, 5, 12, 8, 1]
(x_0=2, m=13, a=5): [2, 10, 11, 3, 2]
(x_0=4, m=13, a=5): [4, 7, 9, 6, 4]


## Prime numbers

You might guess that prime numbers will play an important role in getting long sequences of "random-looking" values. Here is a handy function to test for primes that you will use later.

In [7]:
# `is_prime(n)` lifted from: http://stackoverflow.com/questions/15285534/isprime-function-for-python-language
def is_prime (n):
    if n == 2 or n == 3: return True
    if n < 2 or n%2 == 0: return False
    if n < 9: return True
    if n%3 == 0: return False
    r = int(n**0.5)
    f = 5
    while f <= r:
        if n%f == 0: return False
        if n%(f+2) == 0: return False
        f +=6
    return True

# Some "random" testing.
# For a list of the first 1000 primes, see:
# https://primes.utm.edu/lists/small/1000.txt
assert is_prime (3) == True
assert is_prime (6827) == True
assert is_prime (2**8) == False
assert is_prime (3*5*7) == False

## Finding full-period sequences

For any LCS, the maximum interesting sequence length is $m-1$. A sequence that generates only unique values up to this limit is said to have a _full period_. In building a pseudorandom number generator, we care about full periods because these are the longest sequences we can generate without repeating values; once we repeat values, our random numbers will no longer "look random."

Let's restrict our attention to Lehmer generators ($c=0$). Here are some fun facts about them. For proofs, see Lemmis & Park (2006), a link to which appears at the top of this notebook.

**Theorem (L&P, Thm. 2.1.1, pg. 42).** Given the parameters, $(X_0, m, a)$, a Lehmer sequence obeys the relation,

$$
  X_k \equiv (a^k \cdot X_0) \mod m.
$$

This formula is not a great way to compute $X_k$ but _is_ useful for analysis. In particular, it suggests that

$$\begin{eqnarray}
  X_k & \equiv & [(a^k \bmod m) \cdot (X_0 \bmod m)] \mod m \\
      & \equiv & [(a^k \bmod m) \cdot X_0] \mod m.
\end{eqnarray}$$

In particular, it also holds for $k=m-1$, the maximum possible sequence period. Now suppose $m$ is prime. Then $a \bmod m \neq 0$ and, by [Fermat's Little Theorem](https://en.wikipedia.org/wiki/Fermat%27s_little_theorem), $a^{m-1} \bmod m=1$. In other words, $X_{m-1} = X_0$, meaning the sequence must "wraparound" to $X_0$ for _any_ $X_0$.

**Theorem (L&P, Thm. 2.1.2, pg. 42).** Let $X_0$, $X_1$, $X_2$, $\ldots$ be a Lehmer sequence (increment $c=0$) with a _prime_ modulus $m$ and multiplier $a$. Then, there exists an integer $p \leq m-1$ such that $X_0$, $X_1$, $\ldots$, $X_{p-1}$ are unique and

$$
  X_{i+p} = X_0.
$$

In other words, the sequence is periodic starting at $X_0$ with some _fundamental period_, $p$. Additionally, $(m-1) \mod p = 0$.

> What does this theorem imply? First, for _any_ initial seed, the sequence must repeat with period $p$. Secondly, either $p=m-1$ or $p$ divides $m-1$. Lastly, all the possible periodic sequences for this value of $m$ are disjoint. Our usual goal will be to choose $a$ and $m$ so that $p=m-1$, i.e., the sequence has a _full period_.

**Mersenne primes.** One convenient subset of useful primes are the _Mersenne primes_, which are prime numbers of the form $2^k - 1$ where $k$ is a positive integer. These are useful because there are cheap ways to compute operations modulo $m$ when $m$ is a Mersenne prime by exploiting the binary representation of integers. (We will skip the details, but if you are interested, see the Knuth reading.)

**Exercise.** What is the value of $k$ for the largest Mersenne prime less than 256?

> Answer: ??

Suppose we choose $m$ to be prime, and further suppose that we consider only Lehmer sequences, i.e., $c=0$. How should we choose $a$? The following theorem will help.

**Theorem (L&P, Thm. 2.1.4, pg. 45).** If $a$ is any full-period multiplier of a Lehmer generator with _prime_ modulus $m$, then $a^k \bmod m$ is also a full-period multiplier **if and only if** $k$ and $m-1$ are relatively prime.

In other words, if you can find any full-period generator, you can find them all.

**Exercise.** Given a full-period multiplier for a Lehmer generator with prime modulus $m$, write a function to determine all other full-period multipliers.

> Hint: To determine if two integers are relatively prime, use `math.gcd()`.

In [ ]:
from math import gcd # Should prove handy!

def find_full_period_multipliers (m, a):
    """
    Given a full-period multiplier `a` for a Lehmer generator with
    _prime_ modulus `m`, returns a list of all full-period generators.
    """
    assert is_prime (m)
    A = [a] # At least one, by assumption
    
    # @YOUSE
    ...

    # Done; return
    return A

In [ ]:
# Test code: Check return by brute force
M, A_KNOWN = 13, 6
assert is_prime (M)

A_full = find_full_period_multipliers (M, A_KNOWN)
print ("Computed full-period multipliers:", A_full)

print ("\nScanning all possible multipliers...")
for a in range (2, M):
    X = g_seq (1, M, a)
    assert X[0] == X[-1]
    if len (X) == M:
        print (a, "is a full-period multiplier.")
        assert a in A_full